## Music Genre Classifier With TensorFlow

The objective of this project is to classify 30 sec wav files by genre using a TensorFlow CNN model. The GTZAN dataset can be found here:

https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification

To classify audio samples, we will preprocess them by calculating their MFCC, which is a temporal representation of the energy for each perceived frequency band. In this case, we are choosing 13 bands.

In [1]:
import os
import json
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

import wandb
from wandb.keras import WandbCallback

In [3]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'acc',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [5, 10, 15]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'dropout': {
            'values': [0.3, 0.4, 0.5]
        },
        'conv_layer_size': {
            'values': [16, 32, 64]
        },
        #'weight_decay': {
        #    'values': [0.0005, 0.005, 0.05]
        #},
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        #'optimizer': {
        #    'values': ['adam', 'nadam', 'sgd', 'rmsprop']
        #},
        #'activation': {
        #    'values': ['relu', 'elu', 'selu', 'softmax']
        #}
    }
}

In [4]:
sweep_id = wandb.sweep(sweep_config, entity="msaintfelix", project="Sweeps-Genre-Classifier")

Create sweep with ID: e7xdbkpy
Sweep URL: https://wandb.ai/msaintfelix/Sweeps-Genre-Classifier/sweeps/e7xdbkpy


In [5]:
# Dataset location
SOURCE_PATH = 'Dataset/genres_original/'

# Path to labels and processed data file, json format.
JSON_PATH = 'data.json'

# Sampling rate.
sr = 22050

# Let's make sure all files have the same amount of samples, pick a duration right under 30 seconds.
TOTAL_SAMPLES = 29 * sr

# The dataset contains 999 files. Lets make it bigger. 
# X amount of slices => X times more training examples.
NUM_SLICES = 10
SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

In [6]:
def preprocess_data(source_path, json_path):

    # Let's create a dictionary of labels and processed data.
    mydict = {
        "labels": [],
        "mfcc": []
        }

    # Let's browse each file, slice it and generate the 13 band mfcc for each slice.
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(source_path)):

        for file in filenames:
            song, sr = librosa.load(os.path.join(dirpath, file), duration=29)

            for s in range(NUM_SLICES):
                start_sample = SAMPLES_PER_SLICE * s
                end_sample = start_sample + SAMPLES_PER_SLICE
                mfcc = librosa.feature.mfcc(y=song[start_sample:end_sample], sr=sr, n_mfcc=13)
                mfcc = mfcc.T
                mydict["labels"].append(i-1)
                mydict["mfcc"].append(mfcc.tolist())

    # Let's write the dictionary in a json file.    
    with open(json_path, 'w') as f:
        json.dump(mydict, f)
    f.close()

In [7]:
def load_data(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()

    # Let's load our data into numpy arrays for TensorFlow compatibility.
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    print(X.shape)

    return X, y

In [8]:
def prepare_datasets(inputs, targets, split_size):
    
    # Creating a validation set and a test set.
    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=split_size)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size=split_size)
    
    # Our CNN model expects 3D input shape.
    inputs_train = inputs_train[..., np.newaxis]
    inputs_val = inputs_val[..., np.newaxis]
    inputs_test = inputs_test[..., np.newaxis]
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [9]:
def make_prediction(model, X, y, idx):
    
    genre_dict = {
        0 : "blues",
        1 : "classical",
        2 : "country",
        3 : "disco",
        4 : "hiphop",
        5 : "jazz",
        6 : "metal",
        7 : "pop",
        8 : "reggae",
        9 : "rock",
        }
        
    predictions = model.predict(X)
    genre = np.argmax(predictions[idx])
    
    print("\n---Now testing the model for one audio file---\nThe model predicts: {}, and ground truth is: {}.\n".format(genre_dict[genre], genre_dict[y[idx]]))

In [10]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 5,
        'batch_size': 32,
        'weight_decay': 0.0005,
        'learning_rate': 1e-3,
        'activation': 'relu',
        'optimizer': 'nadam',
        'hidden_layer_size': 64,
        'conv_layer_size': 32,
        'dropout': 0.5,
        'momentum': 0.9,
        'seed': 42
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    #preprocess_data(source_path=SOURCE_PATH, json_path=JSON_PATH)
    
    inputs, targets = load_data(json_path=JSON_PATH)
        
    Xtrain, Xval, Xtest, ytrain, yval, ytest = prepare_datasets(inputs, targets, 0.2)

    input_shape = (Xtrain.shape[1], Xtrain.shape[2], 1)
    
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Conv2D(config.conv_layer_size, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(config.conv_layer_size, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(config.conv_layer_size, (2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(config.dropout),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(config.hidden_layer_size, activation='relu'), 
        tf.keras.layers.Dense(len(np.unique(targets)), activation='softmax')
    ])

    # Selection of the optimizer, loss type and metrics for performance evaluation.
    model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=config.learning_rate),
                     loss='sparse_categorical_crossentropy',
                     metrics = ['acc']
                     )

    model.summary()

    # Training the model.
    history = model.fit(Xtrain, ytrain,
                        validation_data=(Xval, yval),
                        epochs=config.epochs,
                        batch_size=config.batch_size,
                        callbacks=[WandbCallback()]
                        )

    # Testing the model on never seen before data.
    make_prediction(model, Xtest, ytest, 24)

In [11]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: mgm4mu2d with config:
wandb: 	batch_size: 32
wandb: 	conv_layer_size: 32
wandb: 	dropout: 0.4
wandb: 	epochs: 15
wandb: 	learning_rate: 0.01
wandb: Currently logged in as: msaintfelix (use `wandb login --relogin` to force relogin)


(9990, 125, 13)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 11, 32)       320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 6, 32)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 6, 32)         128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 4, 32)         9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 2, 32)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 2, 32)         128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 

acc,▁▃▄▅▆▆▆▇▇▇▇▇███
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁
val_acc,▁▅▄▅▇▆▆▇▇▆██▆▇▇
val_loss,█▂▄▂▁▂▂▁▁▂▁▁▂▁▂
acc,0.71219
best_epoch,10
best_val_loss,0.90448
epoch,14
loss,0.88044
val_acc,0.66066


wandb: Agent Starting Run: 8gixp2c4 with config:
wandb: 	batch_size: 64
wandb: 	conv_layer_size: 16
wandb: 	dropout: 0.3
wandb: 	epochs: 15
wandb: 	learning_rate: 1e-05


(9990, 125, 13)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 11, 16)       160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 6, 16)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 6, 16)         64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 4, 16)         2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 2, 16)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 2, 16)         64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 

...

acc,▁▃▄▅▅▆▆▇▇▇▇████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▅▄▄▃▃▂▂▂▂▁▁▁▁
val_acc,▁▄▅▆▆▆▆██▇█▇███
val_loss,█▅▅▃▃▄▃▂▁▃▁▂▁▁▁
acc,0.85828
best_epoch,13
best_val_loss,0.6486
epoch,14
loss,0.40032
val_acc,0.77177


wandb: Agent Starting Run: jza73q8p with config:
wandb: 	batch_size: 64
wandb: 	conv_layer_size: 64
wandb: 	dropout: 0.5
wandb: 	epochs: 15
wandb: 	learning_rate: 3e-05


(9990, 125, 13)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 11, 64)       640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 6, 64)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 6, 64)         256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 4, 64)         36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 2, 64)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 2, 64)         256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 

acc,▁▂▃▄▅▅▆▆▆▇▇▇███
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▅▄▃▃▂▂▂▂▂▁▁▁▁
val_acc,▁▃▄▅▅▆▆▇▇▇▇▇███
val_loss,█▆▅▄▄▃▃▃▂▂▂▂▁▁▁
acc,0.5481
best_epoch,14
best_val_loss,1.16206
epoch,14
loss,1.29423
val_acc,0.58659


wandb: Agent Starting Run: 6oy2t1k8 with config:
wandb: 	batch_size: 64
wandb: 	conv_layer_size: 16
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0003
wandb: Ctrl + C detected. Stopping sweep.
